# Introducción

## El objetivo del presente trabajo práctico es aplicar los conceptos teóricos y práticos aprendidos de la materia a la predicción de una feature en el dataset de lluvia de hamburguesas y a su vez relatar en el notebook el análisis y procedimiento utilizado para poder generar dicha predicción.

In [1]:
import pandas as pd
import numpy as np
import requests
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from pandas_profiling import ProfileReport

In [ ]:
df_1 = pd.read_csv(
    'https://docs.google.com/spreadsheets/d/1gvZ03uAL6THwd04Y98GtIj6SeAHiKyQY5UisuuyFSUs/export?format=csv', low_memory=False
)
df_2 = pd.read_csv(
    'https://docs.google.com/spreadsheets/d/1wduqo5WyYmCpaGnE81sLNGU0VSodIekMfpmEwU0fGqs/export?format=csv', low_memory=False
)
pd.options.display.max_columns = None

df = df_2.merge(df_1, left_on = 'id', right_on = 'id')
df.sort_values(by=['id'], inplace=True, ascending=True)

df_original = df.copy()

df.sample(10)

# Feature engineering
### Buscamos filas duplicadas
### Consideramos a una fila duplicada unicamente cuando se repite el ID de la medicion

In [ ]:
df[df.duplicated(subset=['id'],keep=False)]

### Vemos que no hay duplicados, pero verifiquemos además que el tipo para cada feature es razonable:

In [ ]:
df.dtypes

#### Sabiendo que el pandas trata los strings como 'object', inmediatamente notamos que hay una feature que debería estar marcada como float pero se está tratando como un object: presion_atmosferica_tarde. Por dicho motivo procedemos a transformarla a numérica:

Si hacemos directamente to_numeric(), vemos que encuentra un valor que no puede convertir por tener dos simbolos '.' de decimal, por lo que asumimos que fue un error
al cargar el dato de presion_atmosferica_tarde, y preferimos dejar ese casillero en NaN que eliminar el registro completo:

In [ ]:
df['presion_atmosferica_tarde'].replace('.+\..+\..+', np.nan, inplace=True, regex=True)
df['presion_atmosferica_tarde'] = pd.to_numeric(df['presion_atmosferica_tarde'])

## Variables __categoricas__
1. Barrio 
2. direccion_viento_tarde
3. direccion_viento_temprano
4. rafaga_viento_max_direccion
5. llovieron_hamburguesas_hoy

### Comenzamos por reemplazar los valores nulos en direcciones de viento y ráfagas por una nueva opcion que puede tomar la feature : "Otra direccion"

In [ ]:
df['direccion_viento_tarde'].replace(np.nan, 'Otra direccion', inplace=True)
df['direccion_viento_temprano'].replace(np.nan, 'Otra direccion', inplace=True)
df['rafaga_viento_max_direccion'].replace(np.nan, 'Otra direccion',inplace=True)

### Luego verificamos que ninguna feature categorica tenga valores repetidos o sin sentido por errores de tipado 

In [ ]:
df.barrio.value_counts().to_frame()

In [ ]:
df.direccion_viento_temprano.value_counts().to_frame()

In [ ]:
df.direccion_viento_tarde.value_counts().to_frame()

In [ ]:
df.rafaga_viento_max_direccion.value_counts().to_frame()

In [ ]:
df.llovieron_hamburguesas_hoy.value_counts().to_frame()

#### Vemos que todos los valores son unicos, no hay necesidad de alterar las features y podemos seguir con la conversion a valores numericos

## Variables __numéricas__


### Comenzamos por buscar cuáles columnas tienen missings

In [ ]:
df.fillna(np.nan, inplace = True)
df_nulos = pd.DataFrame({'Cantidad de nulos': df.isnull().sum(), 'Porcentaje de nulos': (df.isnull().mean() * 100)})
df_nulos.sort_values(by=['Porcentaje de nulos'])

Vemos que algunos de los features tienen menos de un 1.5% de NaNs, por lo cual usar la media para rellenarlos no implicaría una disonancia significativa en la información, sinó más bien un suavizado de esos casos vacíos.

In [ ]:
df['temp_max'].replace(np.nan, df['temp_max'].mean(), inplace = True)
df['temp_min'].replace(np.nan, df['temp_min'].mean(), inplace = True)
df['temperatura_temprano'].replace(np.nan, df['temperatura_temprano'].mean(), inplace = True)
df['velocidad_viendo_temprano'].replace(np.nan, df['velocidad_viendo_temprano'].mean(), inplace = True)

df_nulos = pd.DataFrame({'Cantidad de nulos': df.isnull().sum(), 'Porcentaje de nulos': (df.isnull().mean() * 100)})
df_nulos.sort_values(by=['Porcentaje de nulos'])

Para las features con alta cardinalidad de nulos (horas_de_sol, nubosidad_temprano, nubosidad_tarde) analizaremos cada caso en particular para observar la relacion de dichos nulos con la variable target. 
Desconocemos como se computa la nubosidad, y por eso no podemos afirmar que se trata de columnas con missings at random (MAR)}. Es decir, no podemos afirmar que la falta de informacion en nubosidad dependa de horas_de_sol, (lo mismo aplica para nubosidad_temprano y nubosidad_tarde), por ende trataremos estos missings como MCAR completely at random.

# Formulación de Preguntas y Visualización
## Primero nos hacemos una serie de preguntas para comprender y analizar mejor nuestros datos:  

 1. ¿Suponiendo desconocimiento de todas las features excepto el target, que tan probable es que lluevan hamburguesas al dia siguiente?
 2. ¿De qué manera se relaciona el barrio con que lluevan hamburguesas al dia siguiente?
 3. ¿Si llovieron hamburguesas hoy, qué tan probable es que lluevan hamburguesas mañana?
 4. ¿Cómo afecta la feature mm_lluvia_dia al target? ¿Y qué sucede con los mm evaporados?
 5. ¿Hay correlación entre la feature horas_de_sol y la lluvia de hamburguesas del dia siguiente?
 6. ¿Cómo se relacionan la nubosidad_temprano y la nubosidad_tarde con el target?
 7. ¿Las direcciones del viento temprano y tarde, estarán involucradas en la búsqueda del baseline?
 8. ¿Cómo afectan las velocidades de los vientos tempranos y tardes al baseline?
 9. ¿En cuánto al viento, las ráfagas tienen efecto en el target?
 10. ¿Los máximos y mínimos de temperatura, indican algo sobre si llueven o no hamburguesas al dia siguiente?
 11. ¿Tiene relación la humedad con la variable target?
 12. ¿Encontraremos alguna correlacion entre humedad, temperatura y el target, tal cual indicaría el sentido común?
 13. ¿Es observable una correlación entre la presión atmosférica y el target?
 14. ¿Son de utilidad las mediciones donde el target tiene el valor NaN?
 
Buscamos entender cada uno de los datos, cómo se relacionan entre sí y ver que conclusiones podemos obtener. A partir de ellos vamos a determinar los factores que determinan si lloverá al dia siguiente, y verificaremos cuáles features son reelevantes de mantener en el dataset.

### 1.¿Suponiendo desconocimiento de todas las features excepto el target, que tan probable es que lluevan hamburguesas al dia siguiente?

In [ ]:
plt.figure(figsize=(5.4, 3.8), dpi=150)
labels = ['No', 'Si']
sizes = df['llovieron_hamburguesas_al_dia_siguiente'].value_counts()
explode = (0, 0.1)
plt.title('Llueven hamburguesas al dia siguiente')
plt.pie(sizes, explode=explode, data = df['llovieron_hamburguesas_al_dia_siguiente'],labels = labels ,autopct='%1.1f%%',
        shadow=True, startangle=90)
plt.show()

Podemos corroborar, que desconociendo el resto de features, para el 77.6% de los registros en el dataset al dia siguiente no llueven hamburguesas. Esto quiere decir que si armaramos una baseline que simplemente devuelva siempre "NO", el accuracy ya sería de ese porcentaje. Sin embargo, no solo es un accuracy que no nos satisface, sinó que resulta en algo poco reutilizable que desperdicia el resto de información.

### 2. ¿De qué manera se relaciona el barrio con que lluevan hamburguesas al dia siguiente?

In [ ]:
lluvurguesas_por_barrio = df.groupby(by="barrio")['llovieron_hamburguesas_al_dia_siguiente'].value_counts(normalize=True)
lluvurguesas_por_barrio = lluvurguesas_por_barrio.to_frame()
lluvurguesas_por_barrio.rename(columns={'llovieron_hamburguesas_al_dia_siguiente':'porcentaje_por_barrio'},inplace=True)
lluvurguesas_por_barrio.reset_index(inplace=True)

g = sns.catplot(
    data=lluvurguesas_por_barrio, kind="bar",
    x="barrio", y="porcentaje_por_barrio", hue="llovieron_hamburguesas_al_dia_siguiente", order=df["barrio"].value_counts().index,
    ci=None, height=8,aspect=2
)
plt.title("Porcentaje de lluvia de hamburguesas según barrio", fontsize=25)
g.despine(left=True)
plt.ylim(0,1)
plt.xticks(rotation=90)
g.set_axis_labels("Barrio", "Porcentaje de lluvia de hamburguesas",  fontsize=15)
plt.show()

Podemos ver que en ningún barrio es predominante el "SI" para la feature llovieron_hamburguesas_al_dia_siguiente, por lo que no podemos hacer ninguna afirmación positiva para la lluvia de hamburguesas según el barrio. Sin embargo, podemos notar que ciertos barrios como Parque Patricios y Villa Pueryrredón, tienen lluvia de hamburguesas al dia siguiente más seguido de lo normal (un \~30% de las veces), mientras que barrios como Villa Crespo y Villa Santa Rita rondan números mas bajos (\~10%).

### 3.¿Si llovieron hamburguesas hoy, qué tan probable es que lluevan hamburguesas mañana?

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=[6.4 * 2, 4.8], dpi=100)

sns.countplot(data=df, x='llovieron_hamburguesas_hoy', hue='llovieron_hamburguesas_al_dia_siguiente', ax=axes[0])
axes[0].set_title("Cantidad de veces que llovieron hamburguesas \n" +  "al dia siguiente según si llovieron hoy")

sns.barplot(
    data=df.groupby("llovieron_hamburguesas_hoy")
    .llovieron_hamburguesas_al_dia_siguiente.value_counts(normalize=True)
    .rename("llovieron_al_dia_siguiente_prop")
    .reset_index(),
    x='llovieron_hamburguesas_hoy',
    y="llovieron_al_dia_siguiente_prop",
    hue='llovieron_hamburguesas_al_dia_siguiente',
    ax=axes[1],
)
axes[1].set_ylabel("% intragrupo")
axes[1].set_title("Porcentaje para lluvia de hamburguesas\n" +  "al dia siguiente según si llovieron hoy")

plt.show()

Podemos ver que si bien la cantidad de veces que llueven hamburguesas al dia siguiente es prácticamente la misma en ambas mitades del primer gráfico,cuando lo analizamos en porcentajes, vemos que si llovio el dia de hoy, la probabilidad de que lluevan hamburguesas al dia siguiente aumenta radicalmente. Concluimos de esta visualización, que si llovieron hamburguesas hoy, hay practicamente un 45% de probabilidades que llueva mañana, mientras que si hoy no llovieron hamburguesas, entonces esa probabilidad es mucho menor, de aproximadamente el 15%

### 4.¿Cómo afecta la feature mm_lluvia_dia al target? ¿Y qué sucede con los mm evaporados?

In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']

plt.hist(df_no_llovio_h_maniana.mm_lluvia_dia, alpha=1, range=[0, 5], label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.mm_lluvia_dia, alpha=0.75, range=[0, 5], label='Llovieron hamburguesas al dia siguiente')
plt.legend()
plt.title('mm lluvia - llovieron hamburguesas al dia siguiente')
plt.ylabel("Cantidad de mediciones")
plt.xlabel("mm de lluvia")
plt.yscale('log')
plt.show()


Se puede observar que si tomamos en cuenta las distintas escalas del eje de las ordenadas (cantidad de dias donde llovieron o no hamburguesas), entonces la distribución de los mm de lluvia es muy similar en ambas, por lo que no nos llevaria a ninguna estimación concluyente del target. Veamos si para los mm evaporados de agua en el dia obtenemos una distribución mejor indicadora del target:

In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']

plt.hist(df_no_llovio_h_maniana.mm_evaporados_agua, alpha=1, range=[0, 30], label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.mm_evaporados_agua, alpha=0.75, range=[0, 30], label='Llovieron hamburguesas al dia siguiente')
plt.title('mm evaporados - llovieron hamburguesas al dia siguiente')
plt.ylabel("Cantidad de mediciones")
plt.xlabel("mm evaporados")
plt.yscale('log')
plt.legend()
plt.show()

Podemos observar que las distribuciones en la visualización vuelven a ser  similares tanto para el valor afirmativo del target como en caso contrario, por lo que a priori los mm_evaporados no parecen ser un gran indicador para predecir el mismo. Para considerar una relación con la variable target deberiamos apreciar diferencias más significativas.

### 5.¿Hay correlación entre la feature horas_de_sol y la lluvia de hamburguesas del dia siguiente?


In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']

plt.hist(df_no_llovio_h_maniana.horas_de_sol, range=[0,15], label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.horas_de_sol, alpha=0.75, range=[0,15], label='Llovieron hamburguesas al dia siguiente')
plt.title('Horas de sol - llovieron hamburguesas al dia siguiente')
plt.ylabel("Cantidad de mediciones")
plt.xlabel("Horas de sol")
plt.legend()
plt.show()

Comparando ámbos gráficos, vemos grandes diferencias en las distribuciones de las horas de sol según el valor del target. Cuando al día siguiente llueven hamburguesas, el gráfico de la izquierda nos indica que suele haber en general pocas horas de sol. Caso contrario, en el gráfico de la derecha vemos que la mayoría de dias en los cuales no llueven hamburguesas al día siguiente se agrupan a partir de aproximadamente 8 horas de sol.

Sin embargo, debemos analizar que sucede cuando horas de sol tiene missings. Dado que es una feature con una cardinalidad muy grande de valores nulos, amerita un analisis en detalle.

In [ ]:
fig =  plt.figure(figsize=(5.4, 3.8), dpi=150)\

df_horas_sol_desconocidas = df[(df['horas_de_sol'].isna())]

total_dias_lluvia = sum(df_horas_sol_desconocidas.llovieron_hamburguesas_al_dia_siguiente == 'si')
total_dias_sin_lluvia = sum(df_horas_sol_desconocidas.llovieron_hamburguesas_al_dia_siguiente == 'no')

sns.countplot(data=df_horas_sol_desconocidas, x='llovieron_hamburguesas_al_dia_siguiente', hue='llovieron_hamburguesas_al_dia_siguiente')
plt.title("Cantidad de dias que llovieron hamburguesas al día \n siguiente para horas de sol no conocidas")
plt.xlabel("Llovieron hamburguesas al día siguiente")
plt.axhline(y=total_dias_sin_lluvia, color="C0", label=f"{total_dias_sin_lluvia} dias", linestyle='--')
plt.axhline(y=total_dias_lluvia, color="C1", label=f"{total_dias_lluvia} dias",  linestyle='--')
plt.legend(loc='best')
plt.show()

Del grafico de barras vemos que aproximadamente el 80% de los casos en los que no conocemos las horas de sol, tampoco llovieron hamburguesas al dia siguiente. Este dato podría ser tenido en cuenta a la hora de definir que decisión deberia tomar nuestra baseline.

### 6. ¿Cómo se relacionan la nubosidad_temprano y la nubosidad_tarde con el target?



In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']
rango_nubosidad_temp = [0,9]

plt.hist(df_no_llovio_h_maniana.nubosidad_temprano, range=rango_nubosidad_temp, label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.nubosidad_temprano, alpha=0.75, range=rango_nubosidad_temp, label='Llovieron hamburguesas al dia siguiente')
plt.title('nubosidad temprano - llovieron hamburguesas al dia siguiente')
plt.ylabel("Cantidad de mediciones")
plt.xlabel("Nubosidad temprano")
plt.yscale('log')
plt.legend()
plt.show()


Vemos que la distribución en ambos gráficos es muy diferente, por lo que parece interesante considerar los dias de alta nubosidad temprana en búsqueda de predecir target. Pasemos a analizar la nubosidad a la tarde:

In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']
rango_nubosidad_tarde = [0,9]

plt.hist(df_no_llovio_h_maniana.nubosidad_tarde, range=rango_nubosidad_tarde, label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.nubosidad_tarde,alpha=0.75, range=rango_nubosidad_tarde, label='Llovieron hamburguesas al dia siguiente')
plt.title('Nubosidad - llovieron hamburguesas al dia siguiente')
plt.ylabel("Cantidad de mediciones")
plt.xlabel("Nubosidad tarde")
plt.yscale('log')
plt.legend()
plt.show()

Para la nubosidad a la tarde, la visualización resulta muy similar al caso de la nubosidad temprana, tanto para cuando llueven hamburguesas al dia siguiente como cuando no lo hace.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(11,3.8), dpi=125)

df_nubosidad_temprana_desconocida = df[(df['nubosidad_temprano'].isna())]
total_dias_lluvia_nubosidad_temprana_desc = sum(df_nubosidad_temprana_desconocida.llovieron_hamburguesas_al_dia_siguiente == 'si')
total_dias_sin_lluvia_nubosidad_temprana_desc = sum(df_nubosidad_temprana_desconocida.llovieron_hamburguesas_al_dia_siguiente == 'no')

sns.countplot(data=df_nubosidad_temprana_desconocida, x='llovieron_hamburguesas_al_dia_siguiente', hue='llovieron_hamburguesas_al_dia_siguiente', ax=axes[0])
axes[0].set_title("Cantidad de dias que llovieron hamburguesas al día \n siguiente para nubosidad temprano desconocida")
axes[0].set_xlabel("Llovieron hamburguesas al día siguiente")
axes[0].axhline(y=total_dias_lluvia_nubosidad_temprana_desc, color="C1", label=f"{total_dias_lluvia_nubosidad_temprana_desc} dias",  linestyle='--')
axes[0].axhline(y=total_dias_sin_lluvia_nubosidad_temprana_desc, color="C0", label=f"{total_dias_sin_lluvia_nubosidad_temprana_desc} dias", linestyle='--')
axes[0].legend(loc='best')

df_nubosidad_tarde_desconocida = df[(df['nubosidad_tarde'].isna())]
total_dias_lluvia_nubosidad_tarde_desc = sum(df_nubosidad_tarde_desconocida.llovieron_hamburguesas_al_dia_siguiente == 'si')
total_dias_sin_lluvia_nubosidad_tarde_desc = sum(df_nubosidad_tarde_desconocida.llovieron_hamburguesas_al_dia_siguiente == 'no')

sns.countplot(data=df_nubosidad_tarde_desconocida, x='llovieron_hamburguesas_al_dia_siguiente', hue='llovieron_hamburguesas_al_dia_siguiente',ax=axes[1])
axes[1].set_title("Cantidad de dias que llovieron hamburguesas al día \n siguiente para nubosidad tarde desconocida")
axes[1].set_xlabel("Llovieron hamburguesas al día siguiente")
axes[1].axhline(y=total_dias_lluvia_nubosidad_tarde_desc, color="C1", label=f"{total_dias_lluvia_nubosidad_tarde_desc} dias",  linestyle='--')
axes[1].axhline(y=total_dias_sin_lluvia_nubosidad_tarde_desc, color="C0", label=f"{total_dias_sin_lluvia_nubosidad_tarde_desc} dias", linestyle='--')
axes[1].legend(loc='best')

plt.show()

Analogamente al caso de las horas de sol desconocidas, cuando la nubosidad es desconocida (practicamente idéntico para ambas),en el 80% de los casos no llovieron hamburguesas al dia siguiente. También sera tenido en cuenta este dato a la hora de predecir el target


### 7. ¿Las direcciones del viento temprano y tarde, estarán involucradas en la búsqueda del baseline?

In [ ]:
lluvurguesas_viento_temprano = df.groupby(by="direccion_viento_temprano")['llovieron_hamburguesas_al_dia_siguiente'].value_counts(normalize=True)

lluvurguesas_viento_temprano = lluvurguesas_viento_temprano.to_frame()
lluvurguesas_viento_temprano.rename(columns={'llovieron_hamburguesas_al_dia_siguiente':'porcentaje_viento_temprano'},inplace=True)
lluvurguesas_viento_temprano.reset_index(inplace=True)

g = sns.catplot(
    data=lluvurguesas_viento_temprano, kind="bar",
    x="direccion_viento_temprano", y="porcentaje_viento_temprano", hue="llovieron_hamburguesas_al_dia_siguiente", order=df["direccion_viento_temprano"].value_counts().index,
    ci=None, height=8,aspect=2
)
plt.title("Porcentaje de lluvia de hamburguesas según dirección del viento temprano", fontsize=18)
g.despine(left=True)
plt.ylim(0,1)
plt.xticks(rotation=90)
g.set_axis_labels("Dirección del viento temprano", "Porcentaje de lluvia de hamburguesas",  fontsize=15)
plt.show()

No parece que la dirección del viento temprano tenga una influencia tajante en el comportamiento del target. Vemos que todas las direcciones tienen la mayoría rotunda de no llover hamburguesas al dia siguiente. Probemos ahora qué sucede con la dirección del viento a la tarde.

In [ ]:
lluvurguesas_viento_tarde = df.groupby(by="direccion_viento_tarde")['llovieron_hamburguesas_al_dia_siguiente'].value_counts(normalize=True)

lluvurguesas_viento_tarde = lluvurguesas_viento_tarde.to_frame()
lluvurguesas_viento_tarde.rename(columns={'llovieron_hamburguesas_al_dia_siguiente':'porcentaje_viento_tarde'},inplace=True)
lluvurguesas_viento_tarde.reset_index(inplace=True)

g = sns.catplot(
    data=lluvurguesas_viento_tarde, kind="bar",
    x="direccion_viento_tarde", y="porcentaje_viento_tarde", hue="llovieron_hamburguesas_al_dia_siguiente", order=df["direccion_viento_tarde"].value_counts().index,
    ci=None, height=8,aspect=2
)
plt.title("Porcentaje de lluvia de hamburguesas según dirección del viento tardío", fontsize=18)
g.despine(left=True)
plt.ylim(0,1)
plt.xticks(rotation=90)
g.set_axis_labels("Dirección del viento tardío", "Porcentaje de lluvia de hamburguesas",  fontsize=15)
plt.show()

Al igual que con el viento temprano, el viento a la tarde no parece tampoco influenciar el target de manera determinante como para volverlo parte de nuestra predicción.

### 8. ¿Cómo afectan las velocidades de los vientos tempranos y tardes al baseline?

In [ ]:
fig = plt.figure( figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']
rango_velocidad_viento_temp = [0,70]

plt.hist(df_no_llovio_h_maniana.velocidad_viendo_temprano,range=rango_velocidad_viento_temp, label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.velocidad_viendo_temprano, alpha=0.75, range=rango_velocidad_viento_temp, label='Llovieron hamburguesas al dia siguiente')
plt.title('Velocidad viento temprano - llovieron hamburguesas al dia siguiente')
plt.ylabel("Cantidad de mediciones")
plt.xlabel("Velocidad del viento [km/h]")
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure( figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']
rango_velocidad_viento_tarde = [0,70]

plt.hist(df_no_llovio_h_maniana.velocidad_viendo_tarde,range=rango_velocidad_viento_temp, label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.velocidad_viendo_tarde, alpha=0.75, range=rango_velocidad_viento_temp, label='Llovieron hamburguesas al dia siguiente')
plt.title('Velocidad viento tardío - hamburguesas al dia siguiente')
plt.ylabel('Cantidad de mediciones')
plt.xlabel('Velocidad del viento[km/h]')
plt.yscale('log')
plt.legend()
plt.show()

Concluimos que la distribución del viento temprano, así como el viento tardío, son prácticamente idénticas lluevan hamburguesas al dia siguiente o no, por lo que no nos servirán para la predicción.

### 9. ¿En cuánto al viento, las ráfagas tienen efecto en el target?

In [ ]:
lluvurguesas_dir_rafaga = df.groupby(by="rafaga_viento_max_direccion")['llovieron_hamburguesas_al_dia_siguiente'].value_counts(normalize=True)

lluvurguesas_dir_rafaga = lluvurguesas_dir_rafaga.to_frame()
lluvurguesas_dir_rafaga.rename(columns={'llovieron_hamburguesas_al_dia_siguiente':'porcentaje_dir_rafaga'},inplace=True)
lluvurguesas_dir_rafaga.reset_index(inplace=True)

g = sns.catplot(
    data=lluvurguesas_dir_rafaga, kind="bar",
    x="rafaga_viento_max_direccion", y="porcentaje_dir_rafaga", hue="llovieron_hamburguesas_al_dia_siguiente", order=df["rafaga_viento_max_direccion"].value_counts().index,
    ci=None, height=8,aspect=2
)
plt.title("Porcentaje de lluvia de hamburguesas al dia siguiente \n según la dirección de la ráfaga de viento", fontsize=18)
g.despine(left=True)
plt.ylim(0,1)
plt.xticks(rotation=90)
g.set_axis_labels("Dirección de la ráfaga de máximo viento", "Porcentaje de lluvia de hamburguesas al dia siguiente",  fontsize=15)
plt.show()

Vemos que al igual que el propio viento, la dirección de su máxima ráfaga no parece tener un distribución divisiva en cuanto al target. ¿Y qué hay de la velocidad de la ráfaga? Vemos a continuación:

In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

df_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'si']
df_no_llovio_h_maniana = df[df.llovieron_hamburguesas_al_dia_siguiente == 'no']
rango_velocidad_rafaga = [0,120]

plt.hist(df_no_llovio_h_maniana.rafaga_viento_max_velocidad,  range=rango_velocidad_rafaga, label='No llovieron hamburguesas al dia siguiente')
plt.hist(df_llovio_h_maniana.rafaga_viento_max_velocidad, alpha=0.75, range=rango_velocidad_rafaga, label='Llovieron hamburguesas al dia siguiente')
plt.title('Velocidad máxima ráfaga de viento - hamburguesas al dia siguiente')
plt.ylabel('Cantidad de mediciones')
plt.xlabel('Velocidad del viento[km/h]')
plt.yscale('log')
plt.legend()
plt.show()

Adicionalmente, vemos que la velocidad de dicha ráfaga tiene también una distribución similar tanto para los casos en los que llueve al dia siguiente como en los que no, por lo que concluimos finalmente que ninguno de los datos respecto al viento nos da información que vayamos a utilizar para discriminar en el baseline.

 ### 10. ¿Los máximos y mínimos de temperatura, indican algo sobre si llueven o no hamburguesas al dia siguiente?

In [ ]:
plt.figure(dpi=150)
plt.title("Distribución de la temperatura máxima según \n si llovieron hamburguesas o no al día siguiente")
sns.violinplot(
    data=df,
    y='temp_max',
    x='llovieron_hamburguesas_al_dia_siguiente'
#    palette=['#D17049', "#89D15E"],
)
plt.xlabel("Llovieron hamburguesas al día siguiente")
plt.ylabel("Temperatura máxima  [Celcius]")
plt.xticks([False, True], ["No", "Sí"])
plt.show()

In [ ]:
plt.figure(dpi=150)
plt.title("Distribución de la temperatura mínima según \n si llovieron hamburguesas o no al día siguiente")
sns.violinplot(
    data=df,
    y='temp_min',
    x='llovieron_hamburguesas_al_dia_siguiente'
#    palette=['#D17049', "#89D15E"],
)
plt.xlabel("Llovieron hamburguesas al día siguiente")
plt.ylabel("Temperatura mínima  [Celcius]")
plt.xticks([False, True], ["No", "Sí"])
plt.show()

Vemos que ni la temperatura máxima ni la mínima en el día parecen ser determinantes de si llueven o no hamburguesas al día siguiente. La forma del violin es prácticamente la misma se cumpla o no el target.

### 11. ¿Tiene relación la humedad con la variable target?
Para responder esta pregunta analitica, planteamos dos boxplots, uno para cada momento del dia

In [ ]:
plt.figure(dpi=120)
plt.title("Distribucion de la humedad segun si llovio al dia siguiente o no (temprano) ")
sns.boxplot(
    data=df,
    y='humedad_temprano',
    x='llovieron_hamburguesas_al_dia_siguiente'
)
plt.ylabel("Humedad porcentual")
plt.xlabel("Llovieron hamburguesas al dia siguiente")
plt.axhline(y=80, color="blue", label="%80", alpha=0.7)
plt.legend(title="Porcentaje de humedad")
plt.show()

No es observable una gran diferencia en cuanto a las distribuciones de humedad temprana. Se ve como dato notable que la mediana de humedad para cuando llueven hamburguesas al día siguiente esta donde termina el rango intercuartilico de la humedad para los dias que no llovieron hamburguesas, sin embargo no es un dato tan fuerte como para suponer afirmaciones en la baseline. Analizaremos la humedad a la tarde para ver si podemos concluir afirmaciones mas precisas.

In [ ]:
plt.figure(dpi=120)
plt.title("Distribucion de la humedad segun si llovio al dia siguiente o no (tarde)")
sns.boxplot(
    data=df,
    y='humedad_tarde',
    x='llovieron_hamburguesas_al_dia_siguiente'
)
plt.ylabel("Humedad porcentual")
plt.xlabel("Llovieron hamburguesas al dia siguiente")
plt.axhline(y=60, color="blue", label="%60", alpha=0.7)
plt.axhline(y=84, color="red", label="%84", alpha=0.7)
plt.legend(title="Porcentaje de humedad")
plt.xticks([False, True], ["No", "Sí"])
plt.show()

En este caso se ve una diferencia mucho mas clara, ya que donde termina el rango intercuartilico de los casos en los que no llueven hamburguesas al dia siguiente, 
esta proximo al inicio del rango intercuartilico de los casos en los que si llueven al día siguiente. Entonces podemos suponer que si la humedad a la tarde es aproximadamente mayor a 80%, es probable que al dia siguiente lluevan hamburguesas.

### 12. ¿Encontraremos alguna correlacion entre humedad, temperatura y el target, tal cual indicaría el sentido común?
En el clima de nuestra región es intuitivo pensar que para temperaturas y humedades altas, aumenta la probabilidad de que llueva, sin embargo el target no se trata de una lluvia común y corriente si no de una lluvia de hamburguesas y por ende no podemos confiar en la lógica y el sentido común. Debemos realizar un análisis mas riguroso, para lo cual planteamos un scatter plot que relaciona la humedad en el eje X, la temperatura en el eje Y, y la variable target en el color de los puntos, uno para cada momento del día (temprano y tarde).

In [ ]:
fig, ax = plt.subplots(dpi=150)

df_copia = df.copy()
df_copia['llovieron_hamburguesas_al_dia_siguiente'].replace(['no','si'],[0,1],inplace=True)
df_copia = df_copia.dropna(subset = ['llovieron_hamburguesas_al_dia_siguiente'])

sns.scatterplot(
    x='humedad_temprano',
    y='temperatura_temprano',
    hue=df_copia['llovieron_hamburguesas_al_dia_siguiente'].tolist(),
    data=df_copia,
    alpha=0.7,
)
ax.legend(title="Llovieron hamburguesas al dia siguiente")
plt.xlabel("Humedad")
plt.ylabel("Temperatura")
plt.title('Relación humedad - temperatura (temprano)')
plt.show()

In [ ]:
fig, ax = plt.subplots(dpi=150)

df_copia = df.copy()
df_copia['llovieron_hamburguesas_al_dia_siguiente'].replace(['no','si'],[0,1],inplace=True)
df_copia = df_copia.dropna(subset = ['llovieron_hamburguesas_al_dia_siguiente'])

sns.scatterplot(
    x='humedad_tarde',
    y='temperatura_tarde',
    hue=df_copia['llovieron_hamburguesas_al_dia_siguiente'].tolist(),
    data=df_copia,
    alpha=0.7,
)
ax.legend(title="Llovieron hamburguesas al dia siguiente")
plt.xlabel("Humedad")
plt.ylabel("Temperatura")
plt.title('Relación humedad - temperatura (tarde)')
plt.show()

Para ambos momentos del día podemos observar de manera difusa y poco clara una relación lineal inversa entre temperatura y humedad. 

En el gráfico de el momento del día temprano, la mayoria de puntos indican que al día siguiente no llovieron hamburguesas. Solo para humedades muy altas (mas de 80%) y temperaturas de entre 20 y 30 grados celsius vemos una leve predominancia de lluvia de hamburguesas al día siguiente. También vemos casos separados del grupo mencionado para todas las humedades, así que el primer gráfico no nos parece lo suficientemente conclusivo.

En cambio, para el grafio del momento del día tarde, vemos una separación mas clara de los puntos en cuanto al target se refiere, siendo que la mayoría de casos donde el target es positivo se agrupan para humedades altas, de entre 70-100%, y la mayoría de casos donde dicha feature es negativa, se agrupan para valores mucho menores de humedad. 

Esto tiene sentido ya que la tarde esta mucho mas cercana del dia siguiente que la mañana, es decir, si a la mañana hay una humedad alta, esta tiene mas posibilidades de cambiar antes del día siguiente que la humedad de la tarde, dejando de afectar al target.

### 13. ¿Es observable una correlación entre la presión atmosférica y el target?

In [ ]:
plt.figure(dpi=100)
plt.title("Distribución de la presión atmosférica temprana \n si llovieron hamburguesas o no al día siguiente")
sns.violinplot(
    data=df,
    y='presion_atmosferica_temprano',
    x='llovieron_hamburguesas_al_dia_siguiente'
)
plt.xlabel("Llovieron hamburguesas al día siguiente")
plt.ylabel("Presión atmosférica [hPa]")
plt.xticks([False, True], ["No", "Sí"])
plt.show()

In [ ]:
plt.figure(dpi=100)
plt.title("Distribución de la presión atmosférica a la tarde \n si llovieron hamburguesas o no al día siguiente")

df_aux = df.explode('presion_atmosferica_tarde')
df_aux['presion_atmosferica_tarde'] = df_aux['presion_atmosferica_tarde'].astype('float')

sns.violinplot(
    data=df_aux,
    y='presion_atmosferica_tarde',
    x='llovieron_hamburguesas_al_dia_siguiente'
)
plt.xlabel("Llovieron hamburguesas al día siguiente")
plt.ylabel("Presión atmosférica [hPa]")
plt.xticks([False, True], ["No", "Sí"])
plt.show()

### 14. ¿Son de utilidad las mediciones donde el target tiene el valor NaN?

Recordando como se ve a continuación, que el porcentaje de missings en la variable target es relativamente bajo, es válido plantearnos si seria conveniente droppear del dataset los registros que tengan un NaN en la variable target.

In [ ]:
df_con_nulos = df_nulos[df_nulos['Cantidad de nulos'] > 0]
df_con_nulos.sort_values(by=['Porcentaje de nulos']).head()

Como podemos ver, llovieron_hamburguesas_al_dia_siguiente tiene missings en un ~2.3% de los registros del dataset, por lo que no sería a primera vista muy grave droppear aquellas mediciones con el target en NaN. Sin embargo, también es de importancia tener en cuenta que a la hora de predecir el target en nuestro algoritmo no tendremos con qué comparar la estimación de dicho algoritmo si la feature pertinente se encuentra en NaN. Esto nos resulta un importante indicativo de que terminaremos ignorando en el baseline las mediciones con missings en el target. 

Aun así, vamos a comparar para estar seguros las distribuciones de las feature humedad_tarde, horas_sol y nubosidad_tarde cuando el target está en NaN contra cuando no. Ya que estas features son las que nosotros consideramos con mejor potencial de predicción, si no tienen un cambio radical de comportamiento cuando el target tiene missings, entonces procederemos a droppear dichos registros.

In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

df_target_missings = df[(df['llovieron_hamburguesas_al_dia_siguiente'].isna())]
df_target_completo = df[(df['llovieron_hamburguesas_al_dia_siguiente'].isna() == False)] 

plt.hist(df_target_completo.humedad_tarde, label='Humedad cuando el target tiene valor')
plt.hist(df_target_missings.humedad_tarde, alpha=0.75, label='Humedad cuando el target es nulo')
plt.yscale("log")

plt.ylabel("Cantidad")
plt.xlabel("Humedad")

plt.legend()
plt.show()

Vemos que a pesar de ser una cantidad mucho menor de registros (lo cuál es esperado habiendo solo un 2.3% de missings en el target), se mantiene la forma de la distribución de humedad a pesar de no conocer el valor de llovieron_hamburguesas_al_dia_siguiente. Comprobemos qué sucede con las horas de sol:

In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

plt.hist(df_target_completo.horas_de_sol, label='Horas de sol cuando el target tiene valor')
plt.hist(df_target_missings.horas_de_sol, alpha=0.75, label='Horas de sol cuando el target es nulo')
plt.yscale("log")

plt.ylabel("Cantidad")
plt.xlabel("Horas de sol")

plt.legend()
plt.show()

En el caso de las horas de sol, tenemos una situación similar donde la forma de la distribución se mantiene independientemente de los missings del target, si bien obviamente la escala sigue siendo muy distinta. Pasemos a analizar la nubosidad:

In [ ]:
fig = plt.figure(figsize=[6.4 * 2, 4.8], dpi=100)

plt.hist(df_target_completo.nubosidad_tarde,range=[0,9], label='Nubosidad a la tarde cuando el target tiene valor')
plt.hist(df_target_missings.nubosidad_tarde,range=[0,9], alpha=0.75, label='Nubosidad a la tarde cuando el target es nulo')
plt.yscale("log")

plt.ylabel("Cantidad")
plt.xlabel("Nubosidad a la tarde")

plt.legend()
plt.show()

En este caso la diferencia de escalas es mucho más grosera, pues podemos ver que para nubosidades muy bajas tenemos una cantidad de registros que ronda los dígitos dobles si el target es nulo, mientras que cuando no es nulo tenemos alrededor de 4000 registros con dicho valor de nubosidad. Otro aspecto interesante, es que las nubosidades altas (>=8) parecen ser raras en ambos casos.

A pesar de esto, sigue sin ser una visualización que nos indique una correlación entre la existencia de un valor concreto del target y la nubosidad, por lo que nuevamente se refuerza nuestra hipótesis, e ignoraremos del dataset aquellos registros donde el target sea nulo.

# Conclusiones de la visualizacion y análisis:
 * La humedad de la tarde parece ser una buena indicadora de la variable target, siendo así que será considerada como feature principal de predicción dentro de nuestra baseline.
 * Las features horas de sol y de nubosidades, a pesar de tener una gran cantidad de missings, fueron destacadas ya que también discriminan de manera clara los casos en los que llueven hamburguesas al día siguiente de los que no.
 * La feature "llovieron_hamburguesas_hoy" también sera considerada para predecir el target ya que cuando su valor es 'si', la probabilidad de que llovieron_hamburguesas_al_dia_siguiente sea positiva aumenta drásticamente con respecto a cuando no.
 * Hay una gran cantidad de features de las cuales no pudimos sacar conclusiones que nos ayuden a predecir el valor de la variable target ya que poseian una distribución equitativa en todos sus valores posibles, para ambos valores del target.

# Baselines posibles
### Comencemos por definir una serie de baselines posibles según nuestro análisis previo, con el objetivo de intentar predecir la variable target.

#### Para esto primero crearemos una función de preprocesamiento, la cuál se encargará de:
    1. Droppear del dataset los registros que tengan un NaN en la variable target, por lo cual no podemos saber si nuestra predicción está bien o no, ni aprender nada de ellos.
    2. Droppear del dataset las filas que tengan todos los siguientes registros en null: humedad_tarde, horas_de_sol, nubosidad_tarde. Esto se debe a que son los más indicativos que vimos durante el análisis y necesitaremos de ellos para formar el baseline.
    3. Droppear del dataset los registros que tengan alrededor de la mitad de campos en NaN, los cuales consideramos mediciones de poca calidad e incompletas.

In [ ]:
def preprocesar_df(df_raw:pd.DataFrame):
    df_raw = df_raw.dropna(subset=['llovieron_hamburguesas_al_dia_siguiente'])
    df_raw = df_raw.dropna(subset=['humedad_tarde', 'horas_de_sol', 'nubosidad_tarde'], how='all')
    df_raw = df_raw.dropna(thresh = len(df_raw.columns) - 10)
    return df_raw

#### Una vez definida la función de preprocesamiento, programamos una serie de diferentes baselines que intuimos superaran el 80% de accuracy al predecir el target:

In [ ]:
def baseline(df_baseline:pd.DataFrame):
    
    llovio_h_hoy = (df_baseline['llovieron_hamburguesas_hoy'] == 'si')
    
    alta_humedad_tarde = (df_baseline['humedad_tarde'] >= 79)
    horas_sol_bajas = (df_baseline['horas_de_sol'] <= 7.5)
    alta_nubosidad_tarde = (df_baseline['nubosidad_tarde'] >= 7.1)
    
    df_baseline['prediccion_llueven_h_al_dia_siguiente'] = 'no'
    df.loc[alta_humedad_tarde, 'prediccion_llueven_h_al_dia_siguiente'] = 'si'
    df.loc[llovio_h_hoy & alta_nubosidad_tarde & horas_sol_bajas, 'prediccion_llueven_h_al_dia_siguiente'] = 'si'
    
    resultado =  df_baseline['prediccion_llueven_h_al_dia_siguiente'].values.tolist()
    df_baseline.drop(columns=['prediccion_llueven_h_al_dia_siguiente']) #remuevo la columna de predicciones para no alterar el df original
    return resultado

def baseline_2(df_baseline:pd.DataFrame):
    
    alta_humedad_tarde = (df_baseline['humedad_tarde'] >= 79)
    
    df_baseline['prediccion_llueven_h_al_dia_siguiente'] = 'no'
    df.loc[alta_humedad_tarde, 'prediccion_llueven_h_al_dia_siguiente'] = 'si'
    
    resultado =  df_baseline['prediccion_llueven_h_al_dia_siguiente'].values.tolist()
    df_baseline.drop(columns=['prediccion_llueven_h_al_dia_siguiente'])
    return resultado

def baseline_3(df_baseline:pd.DataFrame):
    
    llovio_h_hoy = (df_baseline['llovieron_hamburguesas_hoy'] == 'si')
    alta_humedad_tarde = (df_baseline['humedad_tarde'] >= 79)
    alta_nubosidad_tarde = (df_baseline['nubosidad_tarde'] >= 7.1)

    df_baseline['prediccion_llueven_h_al_dia_siguiente'] = 'no'
    df.loc[alta_humedad_tarde, 'prediccion_llueven_h_al_dia_siguiente'] = 'si'
    df.loc[llovio_h_hoy & alta_nubosidad_tarde , 'prediccion_llueven_h_al_dia_siguiente'] = 'si'
    
    resultado =  df_baseline['prediccion_llueven_h_al_dia_siguiente'].values.tolist()
    df_baseline.drop(columns=['prediccion_llueven_h_al_dia_siguiente']) #remuevo la columna de predicciones para no alterar el df original
    return resultado






# Predicción según baselines
### Pasamos entonces a probar las funciones baselines definidas y a comparar las accuracies resultantes:

Para el primer baseline, que tiene en cuenta principalmente la feature humedad_tarde, y luego también predice que llueve al dia siguiente para aquellas filas donde haya alta nubosidad a la tarde, bajas horas de sol, y haya llovido hamburguesas hoy, la accuracy resulta:

In [ ]:
df = preprocesar_df(df_original)
(baseline(df) == df['llovieron_hamburguesas_al_dia_siguiente']).mean().round(4)

Para el segundo baseline, que se guia únicamente en si hubo una alta humedad a la tarde o no, la accuracy resulta:

In [ ]:
df = preprocesar_df(df_original)
(baseline_2(df) == df['llovieron_hamburguesas_al_dia_siguiente']).mean().round(4)

Finalmente nuestro tercer baseline es muy parecido al primero, con la diferencia de ignorar las horas de sol:

In [ ]:
df = preprocesar_df(df_original)
(baseline_3(df) == df['llovieron_hamburguesas_al_dia_siguiente']).mean().round(4)

### Se observa que en caso de tener que elegir entre nuestros baselines, aquel con un puntaje de accuracy más elevado, entonces se utilizaría el primero. Sin embargo consideramos de alta importancia notar que el segundo baseline es solamente un 0,1% menos preciso, siendo a su vez más simple por necesitar solamente de la feature humedad_tarde para predecir.